In [37]:
# Mount Drive, Data is saved on my google drive account
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import pickle
import os
import ipaddress
import hashlib
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

# Initialize some colors
R = "\033[91m"  # Red text
W = "\033[0m"   # Reset to default
G = "\033[92m"  # Green text

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
# Initialize Root directory and path to pickle
# Using visualzation data since dtypes are already configured
PROJECT_ROOT = '/content/drive/MyDrive/IDS'
PATH = os.path.join(PROJECT_ROOT, "pkl")
DATA_PATH = os.path.join(PROJECT_ROOT, f'{PATH}/v_data.pkl')
print(DATA_PATH)

/content/drive/MyDrive/IDS/pkl/v_data.pkl


In [39]:
# Load Data
try:
  data = pd.read_pickle(DATA_PATH)
  print(f'{G} Data Loaded! {W}')
except:
  print(f'{R} Data Not Found! {W}')
data.info()

 Data Loaded! 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 589530 entries, 0 to 589529
Data columns (total 21 columns):
 #   Column         Non-Null Count   Dtype                     
---  ------         --------------   -----                     
 0   ts             589530 non-null  datetime64[ns, US/Pacific]
 1   uid            589530 non-null  object                    
 2   src_ip         589530 non-null  object                    
 3   src_p          589530 non-null  int64                     
 4   dst_ip         589530 non-null  object                    
 5   dst_p          589530 non-null  int64                     
 6   proto          589530 non-null  object                    
 7   service        589530 non-null  object                    
 8   duration       589530 non-null  float64                   
 9   src_bytes      589530 non-null  float64                   
 10  dst_bytes      589530 non-null  float64                   
 11  conn_state     589530 non-null  objec

# Feature Engineering
- we noticed in our EDA that orig_bytes and orig_pkts, etc.., where found highly corelatted thus to reduce the complexity of our model we will combine them


In [40]:
# Lets check our corr map again
def drop_categorical(df):
  temp = df.drop(columns=['src_ip', 'uid', 'dst_ip', 'proto', 'service', 'conn_state', 'local_orig', 'local_resp', 'history', 'ts'])
  return temp
n_data = drop_categorical(data)

n_data

,src_p,dst_p,duration,src_bytes,dst_bytes,missed_bytes,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,ip_proto
0,65430,443,0.176656,0.0,228408.0,0,34,1876,169,237196,6
1,65289,443,0.089289,24.0,65.0,0,6,300,5,325,6
2,65430,443,0.213978,1544.0,156813.0,0,29,3088,120,161605,6
3,52768,53,0.019330,33.0,83.0,0,1,81,1,131,17
4,55558,53,0.017523,33.0,61.0,0,1,81,1,109,17
...,...,...,...,...,...,...,...,...,...,...,...
589525,1,0,156.840116,8290.0,0.0,0,97,12946,0,0,58
589526,135,136,838.535921,840.0,560.0,0,35,2520,35,2240,58
589527,136,135,871.438452,576.0,384.0,0,36,2304,16,1152,58
589528,3,1,892.440025,122608.0,0.0,0,1703,170292,0,0,1


## Pre Processing
- one hot encode everything but UID, and History
- we will hash UID and History

In [41]:
 # encoded data
 # Missed Bytes is mostly 0's we will drop it
 # f_data['missed_bytes'].value_counts()

 def preprocessing(df):

  # Feature Engineering
  f_data = data.copy()
  f_data['src_bytes/src_pkts'] = f_data['src_bytes'] + f_data['orig_pkts']
  f_data['dst_bytes/dst_pkts'] = f_data['dst_bytes'] + f_data['resp_pkts']
  f_data.drop(['src_bytes', 'orig_pkts', 'dst_bytes', 'resp_pkts', 'orig_ip_bytes', 'resp_ip_bytes'], axis=1, inplace=True)
  # Everything Looks good!
  # Drop Missed_bytes
  fv2_data = df.drop(columns=['missed_bytes'])
  e_data = f_data.copy()


  # Set ip datatype
  temp1 = []
  for x in e_data['src_ip']:
    try:
      temp1.append(int(ipaddress.IPv4Address(x)) % 65535)
    except:
      temp1.append(int(ipaddress.IPv6Address(x)) % 4294967295 )
  e_data['src_ip'] = pd.Series(temp1)

  temp = []
  for x in e_data['dst_ip']:
    try:
      temp.append(int(ipaddress.IPv4Address(x)) % 65535 )
    except:
      temp.append(int(ipaddress.IPv6Address(x)) % 4294967295 )


    # One Hot Encode
  e_data = pd.get_dummies(e_data, columns=['proto', 'service', 'conn_state', 'local_orig', 'local_resp' ], drop_first=True, dtype=float)

  e_data['dst_ip'] = pd.Series(temp)
  e_data['uid'] = hash(str(e_data['uid'])) % 100000
  e_data['history'] = hash(str(e_data['history'])) % 100000
  e_data['src_ip'] = e_data['src_ip'].astype('uint64')
  e_data['dst_ip'] = e_data['dst_ip'].astype('uint64')
  e_data.drop(columns=['ts'],inplace=True)

  return e_data


e_data = preprocessing(data)



In [42]:
e_data

,uid,src_ip,src_p,dst_ip,dst_p,duration,missed_bytes,history,ip_proto,src_bytes/src_pkts,...,conn_state_RSTRH,conn_state_S0,conn_state_S1,conn_state_S2,conn_state_S3,conn_state_SF,conn_state_SH,conn_state_SHR,local_orig_T,local_resp_T
0,46213,2565,65430,52995,443,0.176656,0,27385,6,34.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,46213,2565,65289,64620,443,0.089289,0,27385,6,30.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,46213,2565,65430,52995,443,0.213978,0,27385,6,1573.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,46213,4265057297,52768,518915418,53,0.019330,0,27385,17,34.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,46213,4265057297,55558,518915418,53,0.017523,0,27385,17,34.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
589525,46213,3998337667,1,2887303159,0,156.840116,0,27385,58,8387.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
589526,46213,3800188861,135,4157567228,136,838.535921,0,27385,58,875.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
589527,46213,3800188861,136,4157567228,135,871.438452,0,27385,58,612.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
589528,46213,32814,3,2565,1,892.440025,0,27385,1,124311.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [43]:
PICKLE_FILE_PATH = os.path.join(PATH, "p_data.pkl")
print(f"\nSaving Dataframe to {PICKLE_FILE_PATH}")
e_data.to_pickle(PICKLE_FILE_PATH)
print("\n Pickled Saved!")


Saving Dataframe to /content/drive/MyDrive/IDS/pkl/p_data.pkl

 Pickled Saved!


In [44]:
with open('preprocess_function.pkl', 'wb') as f:
    pickle.dump(preprocessing, f)